# Challenge 3

Install necessary packages

In [132]:
!pip install google-cloud-aiplatform --quiet

Authenticate Google SDK

In [133]:
!gcloud auth application-default login


You are running on a Google Compute Engine virtual machine.
The service credentials associated with this virtual machine
will automatically be used by Application Default
Credentials, so it is not necessary to use this command.

If you decide to proceed anyway, your user credentials may be visible
to others with access to this virtual machine. Are you sure you want
to authenticate with your personal account?

Do you want to continue (Y/n)?  y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=ExxhCyQ2pBr1MiqX8qRYWzJZ5EznoC&prompt=consent&token_

Importing necessary packages

In [134]:
from google import genai
from google.genai import types
import unittest
import json
import vertexai
from vertexai.preview.evaluation import notebook_utils
from vertexai.evaluation import (
    EvalTask,
    PointwiseMetric,
    PairwiseMetric,
    PointwiseMetricPromptTemplate,
    PairwiseMetricPromptTemplate,
    MetricPromptTemplateExamples,
)
from vertexai.generative_models import (
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
)
import pandas as pd
import plotly.graph_objects as go
from IPython.display import HTML, Markdown, display
import uuid

Initializing Constants

In [135]:
project_id="qwiklabs-gcp-03-f951e059a60d"
model_name="gemini-2.5-pro-preview-06-05"
location="global"
vertex_ai_location="us-central1"

Initializing Vertex AI

In [136]:
vertexai.init(project=project_id, location=vertex_ai_location)

Initializing GenAI client for LLM response

In [137]:
client = genai.Client(
    vertexai=True,
    project=project_id,
    location=location,
)

# Classify a text into a category

Creating a classify function that intakes user's msg and classfies it amoung 4 category (Employment/ General Information/ Emergency Services/Tax Related). It returns the category label.

In [138]:
def classify(msg):
  # initialize variables
  contents=[types.Content(role="user", parts=[types.Part.from_text(text=msg)])]
  model=model_name

  si_text1 = """
        You are a classifier that places questions into one of four categories:
        1. Employment
        2. General Information
        3. Emergency Services
        4. Tax Related.

        Classify the following question into one of the categories above.
        Only output the category label."""
  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 1,
    seed = 0,
    max_output_tokens = 65535,
    safety_settings = [],
    system_instruction=[types.Part.from_text(text=si_text1)],
    thinking_config=types.ThinkingConfig(
      thinking_budget=-1,
    ),
  )


  #Generated response chunk
  response_chunks = client.models.generate_content_stream(
      model=model,
      contents=contents,
      config=generate_content_config
  )

  #Return Built Full Response from the chunks
  full_response = ""
  for chunk in response_chunks:
    full_response += chunk.text

  return full_response

Test if it returns correctly

In [139]:
print(classify("Where should I file my income returns?"))

Tax Related.


Test above code with multiple cases foreach category

In [140]:

class test_classify(unittest.TestCase):

    def test_classification_Employment(self):
      section = classify("What documents do I need to submit with my application for a municipal job opening?")
      self.assertEqual(section, "Employment")

    def test_classification_General_Information(self):
      section = classify("What are the operating hours for the local post office?")
      self.assertEqual(section, "General Information")

    def test_classification_Emergency_Services(self):
      section = classify("Who should I call if there's a gas leak in my neighborhood?")
      self.assertEqual(section, "Emergency Services")

    def test_classification_Tax_Related(self):
      section = classify("Where can I get help preparing my annual income tax return?")
      self.assertEqual(section, "Tax Related")

unittest.main(argv=[''], verbosity=2, exit=False)

test_classification_Emergency_Services (__main__.test_classify.test_classification_Emergency_Services) ... ok
test_classification_Employment (__main__.test_classify.test_classification_Employment) ... ok
test_classification_General_Information (__main__.test_classify.test_classification_General_Information) ... ok
test_classification_Tax_Related (__main__.test_classify.test_classification_Tax_Related) ... ok
test_govt_aanouncement_1 (__main__.test_govt_announement.test_govt_aanouncement_1) ... ok
test_govt_aanouncement_2 (__main__.test_govt_announement.test_govt_aanouncement_2) ... 

{'metric': 'YES', 'reason': ''}


ok

----------------------------------------------------------------------
Ran 6 tests in 62.913s

OK


{'metric': 'NO', 'reason': "The post fails the Veracity and Actionability criteria. It uses placeholders like '[Insert Date]' and '[Insert Tomorrow's Date]' instead of providing specific, concrete information. A public announcement must be complete and verifiable to be of any use to the public; as presented, this is an unusable template, not a valid communication."}


# Create a Governemnt announcement content generation function.

It accepts a gist of the announement and frames the announcement which can be posted on social media. It returns the post content as string

In [141]:
# Creating a system instruction for generating the content
post_generation_system_instruction = """
          You are a social media content creator for a government agency. Your goal is to craft engaging, impactful, and clear social media posts for official announcements.

        Your task is to generate a social media post.

        The post must include:

        1. A strong, attention-grabbing opening statement.
        2. Clear and concise information about the announcement.
        3. A call to action or important instruction.
        4. Relevant and impactful emojis to enhance the message and engagement.
        5. At least two relevant hashtags.
        6. tone that is authoritative yet approachable, prioritizing public safety or well-being where applicable.
        7. Add Emojis whereever necessary

        Make sure below evaluation criteria are met:

        1. Accessibility: Is the message effortlessly grasped by all members of the public, including those with limited literacy or who speak English as a second language?
        2. Tone & Register: Does the emotional quality and formality of the post align perfectly with its subject matter (e.g., solemn for crises, respectful for service disruptions, neutral for general advisories)?
        3. Veracity: Is the information presented accurate, verifiable, and devoid of any speculative or potentially misleading statements?
        4. Actionability (if relevant): For time-sensitive or critical situations, does the post effectively guide the audience on what immediate action to take?
        5. Brevity: Is the post optimally concise, ideally fitting within short-form platform constraints (e.g., under 280 characters for X/Twitter)?
        6. Engagement Elements: Are hashtags purposeful and sparse? Are any mentioned entities or accounts tagged appropriately?
        7. Formal Language: Does the content steer clear of specialized jargon, political opinions, informal slang, or promotional language?
        8. Presentation: Is capitalization and punctuation handled correctly, and are emojis used sparingly and effectively to enhance, not distract from, the message?

        Give only the best options if there are multiple options.

        Only output the post.
        """

def generate_govt_content(msg):
  global post_generation_system_instruction

  # initialize variables
  contents=[types.Content(role="user", parts=[types.Part.from_text(text=msg)])]
  model=model_name


  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 1,
    seed = 0,
    max_output_tokens = 65535,
    safety_settings = [],
    system_instruction=[types.Part.from_text(text=post_generation_system_instruction)],
    thinking_config=types.ThinkingConfig(
      thinking_budget=-1,
    ),
  )


  #Generated response chunk
  response_chunks = client.models.generate_content_stream(
      model=model,
      contents=contents,
      config=generate_content_config
  )

  #Return Built Full Response from the chunks
  full_response = ""
  for chunk in response_chunks:
    full_response += chunk.text

  return full_response

Test the generated content

In [142]:
print(generate_govt_content("tomorrow heavy rainfall, leave for all school, college and private organisations, people dont come out"))

⚠️ PUBLIC SAFETY ALERT ⚠️

Due to forecasts of heavy rainfall ⛈️, all schools, colleges, and organizations will be closed tomorrow.

For your safety, please stay home and avoid all non-essential travel. 🏡

#WeatherAlert #PublicSafety


# Evaluation of the generated content by another LLM call

Since the response of the content is not deterministic, it cannot be directly asserted with a value, so I am making another LLM call to determine if the content genreated matches few metrics.

The key metrics are stored in form on system instrcutions.

The response of the function is in JSON
```json
{
  "metric": "YES/NO", //based on content if it matches the expectation it returns "YES" or else returns "NO"
  "reason": "If no, reason is given" // The reason for NO is provided
}
```

In [143]:
evaluation_post_system_instruction = """
      Role: You are an expert public relations officer tasked with validating social media communications from a government entity.

      Objective: Critically assess the provided social media post, which serves as a public announcement (e.g., for weather alerts, facility closures, public holidays, or community updates).

      Evaluation Criteria:

        1. Accessibility: Is the message effortlessly grasped by all members of the public, including those with limited literacy or who speak English as a second language?
        2. Tone & Register: Does the emotional quality and formality of the post align perfectly with its subject matter (e.g., solemn for crises, respectful for service disruptions, neutral for general advisories)?
        3. Veracity: Is the information presented accurate, verifiable, and devoid of any speculative or potentially misleading statements?
        4. Actionability (if relevant): For time-sensitive or critical situations, does the post effectively guide the audience on what immediate action to take?
        5. Engagement Elements: Are hashtags purposeful and sparse? Are any mentioned entities or accounts tagged appropriately?
        6. Formal Language: Does the content steer clear of specialized jargon, political opinions, informal slang, or promotional language?
        7. Presentation: Is capitalization and punctuation handled correctly, and are emojis used sparingly and effectively to enhance, not distract from, the message?
      Output "YES" if the post satisfies all criteria; otherwise, output "NO" as metric.

      Always output as JSON {"metric": "YES/NO", "reason":"If NO, provide the reason here, or else leave empty"}
  """

def evaulaute_post(announcement):
  global evaluation_post_system_instruction
  # initialize variables
  contents=[types.Content(role="user", parts=[types.Part.from_text(text=announcement)])]
  model=model_name

  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 1,
    seed = 0,
    max_output_tokens = 65535,
    safety_settings = [],
    system_instruction=[types.Part.from_text(text=evaluation_post_system_instruction)],
    thinking_config=types.ThinkingConfig(
      thinking_budget=-1,
    ),
  )


  #Generated response chunk
  response_chunks = client.models.generate_content_stream(
      model=model,
      contents=contents,
      config=generate_content_config
  )

  #Return Built Full Response from the chunks
  full_response = ""
  for chunk in response_chunks:
    full_response += chunk.text

  cleaned_json_string = full_response.replace("```json", "").replace("```", "").strip()

  try:
    # Attempt to parse the full string as JSON
    parsed_json = json.loads(cleaned_json_string)
    return parsed_json
  except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    print(f"Received string: {full_response}")
    raise ValueError("LLM response was not a valid JSON string.")


# Evaluating the new function

Based on an announcement generation prompt, the announcement content is validated and respective response is printed for evaluation

In [151]:
resp = generate_govt_content("on '19/06/2025' heavy rainfall, floods expected, leave for all school, college and govt organisations, people dont come out, emergency food will be made available")
print(f"[DEBUG] Post: {resp}")

print(f"\nEvaluation: {evaulaute_post(resp)}")


[DEBUG] Post: ⚠️ PUBLIC SAFETY ALERT: 19/06/2025 ⚠️

Due to predictions of heavy rainfall and potential flooding, all schools, colleges, and government offices will be CLOSED on Thursday, 19th June 2025.

Residents are strongly advised to stay home and avoid travel. 🏠 Your safety is the top priority.

Emergency food and support will be available for affected areas. Stay tuned for updates.

#WeatherAdvisory #StaySafe

Evaluation: {'metric': 'YES', 'reason': ''}


Creating unit test case for both positive and negative scenarios

In [152]:
class test_govt_announement(unittest.TestCase):

    def test_govt_aanouncement_1(self):
      resp = generate_govt_content("From '19/06/2025', new railway station is open for operation at Madurai TN India near Periyar bus stand")
      eval = evaulaute_post(resp)
      self.assertEqual(eval["metric"], "YES")

    def test_govt_aanouncement_2(self):
      resp = generate_govt_content("Declare holiday today")
      eval = evaulaute_post(resp)
      self.assertEqual(eval["metric"], "NO")

unittest.main(argv=[''], verbosity=2, exit=False)

test_classification_Emergency_Services (__main__.test_classify.test_classification_Emergency_Services) ... ok
test_classification_Employment (__main__.test_classify.test_classification_Employment) ... ok
test_classification_General_Information (__main__.test_classify.test_classification_General_Information) ... ok
test_classification_Tax_Related (__main__.test_classify.test_classification_Tax_Related) ... ok
test_govt_aanouncement_1 (__main__.test_govt_announement.test_govt_aanouncement_1) ... ok
test_govt_aanouncement_2 (__main__.test_govt_announement.test_govt_aanouncement_2) ... ok

----------------------------------------------------------------------
Ran 6 tests in 57.726s

OK


## 3. Priting Evaluation metrics based on Vertex AI

Designing few posts and its prompts which is assumed to be generated and its evaluation metrics are determined

In [146]:
# Define a test dataset with vague, generic posts for false positive testing
new_posts = [
    {
        "prompt": "A moderate earthquake of magnitude 4.6 was felt in parts of Himachal Pradesh around 3:45 AM today. No immediate damage has been reported.",
        "post": "Mild earthquake (M4.6) shook parts of Himachal Pradesh early this morning at 3:45 AM. No damage reported so far. Stay alert and follow safety protocols. #Earthquake #HimachalPradesh #StaySafe"
    },
    {
        "prompt": "Delhi Metro services on the Blue Line will be affected on June 25 due to scheduled maintenance work between Dwarka and Rajiv Chowk.",
        "post": "Heads up! Delhi Metro Blue Line services will face disruptions on June 25 between Dwarka and Rajiv Chowk due to maintenance. Plan your commute accordingly. #DelhiMetro #ServiceAlert"
    },
    {
        "prompt": "The civic body has announced a power shutdown in parts of Hyderabad’s Madhapur area on June 18 from 9 AM to 2 PM for line upgrades.",
        "post": "⚡ Power cut alert for Madhapur, Hyderabad on June 18 from 9 AM to 2 PM due to upgrade work. Plan ahead! #Hyderabad #PowerShutdown"
    },
    {
        "prompt": "Something unusual might happen tomorrow, but we’re not sure what exactly. Stay aware.",
        "post": "Tomorrow could be different. Or not. Just keep an eye out. 👀 #StayTuned"
    },
    {
        "prompt": "People are saying things. Some are worried, some are not. It’s unclear what’s really going on.",
        "post": "A lot of chatter lately… unclear what it means. But hey, that’s life. 🤷‍♂️ #WhateverHappens"
    }
]

In [147]:
# Convert the list of posts into a DataFrame for evaluation
# Each row includes the system instruction, the prompt as context, and the generated post as response
evaluation_ds = pd.DataFrame([
    {
        "instruction": post_generation_system_instruction,
        "context": f"announcement: {item['prompt']}",
        "response": item["post"],
    } for item in new_posts
])

# Generate a unique ID for this evaluation experiment run
unique_id = uuid.uuid4()

# Create an evaluation task object with the dataset and a set of evaluation metrics
eval_task = EvalTask(
    dataset=evaluation_ds,
    metrics=[
        MetricPromptTemplateExamples.Pointwise.GROUNDEDNESS,
        MetricPromptTemplateExamples.Pointwise.VERBOSITY,
        MetricPromptTemplateExamples.Pointwise.INSTRUCTION_FOLLOWING,
        MetricPromptTemplateExamples.Pointwise.SAFETY
    ],
    experiment=f"annoumnment{unique_id}"
)

# Define the format of the prompt for the evaluation model
prompt_template = (
    "Instruction: {instruction}. Prompt: {context}. Post: {response}"
)


# Run the evaluation on the dataset using the prompt template and experiment name
result = eval_task.evaluate(
      prompt_template=prompt_template,
      experiment_run_name=f"annoumnment{unique_id}"
)
# Store the evaluation result in a list for later use or output
output = []
output.append(result)

# Print the evaluation result(s)
print(output)

INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'prompt_template': 'Instruction: {instruction}. Prompt: {context}. Post: {response}'}
INFO:vertexai.evaluation._evaluation:Assembling prompts from the `prompt_template`. The `prompt` column in the `EvalResult.metrics_table` has the assembled prompts used for model response generation.
INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 20 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 20/20 [00:02<00:00,  7.03it/s]
INFO:vertexai.evaluation._evaluation:All 20 metric requests are successfully computed.
INFO:vertexai.evaluation._evaluation:Evaluation Took:2.855648786000529 seconds


[EvalResult(summary_metrics={'row_count': 5, 'groundedness/mean': np.float64(1.0), 'groundedness/std': 0.0, 'verbosity/mean': np.float64(-0.4), 'verbosity/std': 0.894427190999916, 'instruction_following/mean': np.float64(3.0), 'instruction_following/std': 1.5811388300841898, 'safety/mean': np.float64(1.0), 'safety/std': 0.0}, metrics_table=                                         instruction  \
0  \n          You are a social media content cre...   
1  \n          You are a social media content cre...   
2  \n          You are a social media content cre...   
3  \n          You are a social media content cre...   
4  \n          You are a social media content cre...   

                                             context  \
0  announcement: A moderate earthquake of magnitu...   
1  announcement: Delhi Metro services on the Blue...   
2  announcement: The civic body has announced a p...   
3  announcement: Something unusual might happen t...   
4  announcement: People are saying things

## Display the evaluated results

In [148]:
# Display the evaluation results in a readable tabular format using a utility function
notebook_utils.display_eval_result(result)

### Summary Metrics

,row_count,groundedness/mean,groundedness/std,verbosity/mean,verbosity/std,instruction_following/mean,instruction_following/std,safety/mean,safety/std
0,5.0,1.0,0.0,-0.4,0.894427,3.0,1.581139,1.0,0.0


### Row-based Metrics

,instruction,context,response,prompt,groundedness/explanation,groundedness/score,verbosity/explanation,verbosity/score,instruction_following/explanation,instruction_following/score,safety/explanation,safety/score
0,\n You are a social media content cre...,announcement: A moderate earthquake of magnitu...,Mild earthquake (M4.6) shook parts of Himachal...,Instruction: \n You are a social medi...,"The response is fully grounded, as it only use...",1.0,"The response is perfectly concise, providing a...",0.0,The response follows the instructions by provi...,4.0,The response is safe as it does not contain an...,1.0
1,\n You are a social media content cre...,announcement: Delhi Metro services on the Blue...,Heads up! Delhi Metro Blue Line services will ...,Instruction: \n You are a social medi...,The response is fully grounded as it is entire...,1.0,"The response is appropriately concise, providi...",0.0,The response includes some of the requested el...,3.0,The response is safe as it does not contain an...,1.0
2,\n You are a social media content cre...,announcement: The civic body has announced a p...,"⚡ Power cut alert for Madhapur, Hyderabad on J...",Instruction: \n You are a social medi...,The response is fully grounded as it only cont...,1.0,The response is just right because it is perfe...,0.0,"The response adheres to all instructions, prov...",5.0,The response is safe as it does not contain an...,1.0
3,\n You are a social media content cre...,announcement: Something unusual might happen t...,Tomorrow could be different. Or not. Just keep...,Instruction: \n You are a social medi...,The response is fully grounded as it only cont...,1.0,The response is appropriately concise and prov...,0.0,"The response is very short and vague, lacking ...",2.0,The response is safe because it does not conta...,1.0
4,\n You are a social media content cre...,announcement: People are saying things. Some a...,A lot of chatter lately… unclear what it means...,Instruction: \n You are a social medi...,The response is fully grounded as it is entire...,1.0,The response is too short and fails to meet th...,-2.0,"The response is extremely poor, it uses the sa...",1.0,"The response does not contain any hate speech,...",1.0
